# Collection of SAS code snippets

## Table of Contents

1. [Syntax sugar for put statements](#Syntax-sugar-for-put-statements)
1. [Word position in list](#Word-position-in-list)
1. [FCMP procedure functions in the FORMAT procedure](#FCMP-procedure-functions-in-the-FORMAT-procedure-in-SAS-9.3)
1. [ANYDTxxx informat to input any date/datetime](#ANYDTxxx-informat-to-input-any-date/datetime)
1. [SQL procedure hidden options: **`_METHOD _TREE`**](#SQL-procedure-hidden-options:-_METHOD-_TREE)

## Syntax sugar for put statements

[⬆️ Back to ToC](#Table-of-Contents)

### SAS Macro new `%put` syntax

New syntax for displaying macro variable name and its value as it is possible in the DATA step:

- `put var=;` – DATA step way
- `%put &=mvar;` – SAS Macro way.

This syntax is available in SAS 9.4.

### Oneliner to print all array value in SAS DATA step

In [1]:
%put &=SYSVLONG;

data _null_;
    array x[*] x1-x12;
    do i = 1 to dim(x);
        x[i] = ranuni(-1);
    end;
    put (x[*]) (=/); /* print out all array values, one value per line */
run;

## Word position in list

[⬆️ Back to ToC](#Table-of-Contents)

The following example shows how modifiers parameter can be used in the [**`FINDW`**](http://documentation.sas.com/?docsetId=lefunctionsref&docsetTarget=p16rdsa30vmm43n1ej4936nwa01t.htm&docsetVersion=9.4&locale=en) function to obtain the index (position) of a word in a list (not character possition):

- **`i`** – ignores the case of the characters
- **`s`** – adds space characters (blank, horizontal tab, vertical tab, carriage return, line feed, and form feed) to the list of characters
- **`e`** – counts the words that are scanned until the specified word is found, instead of determining the character position of the specified word in the string. Fragments of a word are not counted.

In [4]:
%let str = yellow, 88, green, 0;
%let word_pos = %sysfunc(findw(%bquote(&str), green, %str(,), ise));
%put Was found: {%scan(%bquote(&str), &word_pos)} at &word_pos position;

## [FCMP procedure](http://documentation.sas.com/?docsetId=proc&docsetTarget=p10b4qouzgi6sqn154ipglazix2q.htm&docsetVersion=9.4&locale=en) functions in the [FORMAT procedure](http://documentation.sas.com/?docsetId=proc&docsetTarget=p1xidhqypi0fnwn1if8opjpqpbmn.htm&docsetVersion=9.4&locale=en) in SAS 9.3

[⬆️ Back to ToC](#Table-of-Contents)

Based on the [Paper 245-2012 – Using the New Features in PROC FORMAT – Rick Langston](https://support.sas.com/resources/papers/proceedings12/245-2012.pdf).

Example of using [**`PROC FCMP function`**](http://documentation.sas.com/?docsetId=proc&docsetTarget=p10b4qouzgi6sqn154ipglazix2q.htm&docsetVersion=9.4&locale=en) in the [**`FORMAT procedure`**](http://documentation.sas.com/?docsetId=proc&docsetTarget=p1xidhqypi0fnwn1if8opjpqpbmn.htm&docsetVersion=9.4&locale=en) to create flexible formats. The function is defined in PROC FCMP block and later used to format `other` values in the **`h_number`** format. The function checks whether the number is less than 20 and puts it by the following format:

- **`words.`**
- **`best.`** otherwise.

In [5]:
proc fcmp outlib=work.functions.fmts; /* catelog where the functions will be stored */ 
    /* display number in human-readable way */
    function h_number(n) $; /* declare a function taking 1 numeric argument and retuning a string */ 
        length r $30;
        if n <= 20 then do; 
            r = put(n, words.); 
        end;
        else do;
            r = put(n, best.);
        end;
        return (strip(r));
    endsub;
run;

options append=(cmplib=work.functions);

proc format;
    value h_number other = [h_number()];
run;

data h_number;
  input n @@;
  format n_human h_number.;
  n_human = n;
  datalines;
1 1 2 3 5 8 13 21 34 55 89 144
;;;;
run;

proc print data=h_number noobs;
run;

n,n_human
1,one
1,one
2,two
3,three
5,five
8,eight
13,thirteen
21,21
34,34
55,55


## **`ANYDTxxx`** informat to input *any* date/datetime

[⬆️ Back to ToC](#Table-of-Contents)

This snippet is based on the [Paper 2120-2016 – More Hidden Base SAS® Features to Impress Your Colleagues – Peter Crawford](http://support.sas.com/resources/papers/proceedings16/2120-2016.pdf).

- [ANYDTDTEw. Informat](http://documentation.sas.com/?docsetId=leforinforref&docsetTarget=n04jh1fkv5c8zan14fhqcby7jsu4.htm&docsetVersion=9.4&locale=en)
- [ANYDTDTMw. Informat](http://documentation.sas.com/?docsetId=leforinforref&docsetTarget=p1hsn1ji141r4zn0z3xm2dthop6a.htm&docsetVersion=9.4&locale=en)
- [ANYDTTMEw. Informat](http://documentation.sas.com/?docsetId=leforinforref&docsetTarget=n1q732bms1723gn11k9kc61dgy6e.htm&docsetVersion=9.4&locale=en)
- `$ANYDTIFw.` – undocumented. Shows Informat or Form of Input.

In [6]:
data any_testing ;
  infile cards truncover ;
  informat dtm anydtdtm50. dte anydtdte50.
           tme anydttme50.
           anyfmt any2 $anydtif20.;
  format dtm datetime. dte date11. tme time12.1 ;
  input
    @1 cardconts $char50.
    @1 dtm &
    @1 dte &
    @1 TME &
    @1 anyfmt &
    @1 any2 ;
cards;
12/13/2012 15:54:45
12/13/2012 
7
12/13/2012 15:54:45.123
22/12/2012 15:54:45
22/12/2012
;;;;
run;

proc print data=any_testing;
run;

Obs,dtm,dte,tme,anyfmt,any2,cardconts
1,13DEC12:15:54:45,13-DEC-2012,15:54:45.0,ANYDTDTM,MMDDYY,12/13/2012 15:54:45
2,13DEC12:00:00:00,13-DEC-2012,0:00:00.0,MMDDYY,MMDDYY,12/13/2012
3,.,.,.,,,7
4,13DEC12:15:54:45,13-DEC-2012,15:54:45.1,ANYDTDTM,MMDDYY,12/13/2012 15:54:45.123
5,22DEC12:15:54:45,22-DEC-2012,15:54:45.0,ANYDTDTM,DDMMYY,22/12/2012 15:54:45
6,22DEC12:00:00:00,22-DEC-2012,0:00:00.0,DDMMYY,DDMMYY,22/12/2012


## SQL procedure hidden options: **`_METHOD _TREE`**

[⬆️ Back to ToC](#Table-of-Contents)

Undocumented PROC SQL options that displays internals/debug information of a query. This particular example shows that instead of **`sashelp.vcolumn`** view **`DICTIONARY.COLUMNS`** was used. More information about the **`_METHOD`** option can be found [here](http://www.sascommunity.org/wiki/Using_the_PROC_SQL_METHOD_Option) (probably outdated).

In [7]:
proc sql _method _tree noprint;
    select count(*)
    into :sashelp_ds_count
    from sashelp.vcolumn
    where libname="SASHELP";
quit;